# Runtime Test

For conver yolov5 model to trt plan (**yolov5s-2021-07-28.engine**) and build **libmyplugins.so** for your system see instruction: https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5#how-to-run-yolov5s-as-example

Based on this instruction, we wrote a small shell script.

```bash
cd examples/tensorrt/bin
./yolov5_tensorrt.sh
```
Before runing this script, please download install additionaly **opencv-contrib** and **pycuda**

   * Copy the resulting **libmyplugins.so** to the folder from which you will run this notebook
   * Replace the data model **data/models/Detector/yolov5engine/yolov5s-2021-07-28.engine** with the one you just got from running the script
   

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Import all necessary libraries.
import numpy as np
import sys
import time
from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

import warnings
warnings.filterwarnings('ignore')

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)
sys.path.append(os.path.join(NOMEROFF_NET_DIR, "NomeroffNet"))
sys.path.append(os.path.join(NOMEROFF_NET_DIR, "NomeroffNet/tools"))

In [3]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, convertCvZonesRGBtoBGR, getCvZoneRGB, reshapePoints

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [5]:
#!pip3 install pycuda

In [4]:
from examples.tensorrt.YoloV5EngineDetector import Detector

detector = Detector()
detector.load()

[TensorRT] INFO: [MemUsageChange] Init CUDA: CPU +521, GPU +0, now: CPU 3700, GPU 4119 (MiB)
[TensorRT] INFO: Loaded engine size: 24 MB
[TensorRT] INFO: [MemUsageSnapshot] deserializeCudaEngine begin: CPU 3725 MiB, GPU 4119 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +758, GPU +322, now: CPU 5239, GPU 5794 (MiB)


bingding: data (3, 640, 640)
bingding: prob (6001, 1, 1)


[TensorRT] INFO: [MemUsageChange] Init cuDNN: CPU +322, GPU +326, now: CPU 5561, GPU 6120 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 5561, GPU 6104 (MiB)
[TensorRT] INFO: [MemUsageSnapshot] deserializeCudaEngine end: CPU 5561 MiB, GPU 6104 MiB
[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation begin: CPU 5536 MiB, GPU 6104 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 5536, GPU 6112 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuDNN: CPU +1, GPU +8, now: CPU 5537, GPU 6120 (MiB)
[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation end: CPU 5537 MiB, GPU 6152 MiB


In [6]:
from NomeroffNet.OptionsDetector import OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load()

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=15, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(

In [7]:
from NomeroffNet.TextDetector import TextDetector

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    }
})

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
async def test(dirName, fname):
    start_time = time.time()
    img_path = os.path.join(dirName, fname)
    with open(img_path, 'rb') as in_file:
        img = jpeg.decode(in_file.read())
    #img = cv2.imread(img_path)
    image_load_time = time.time() - start_time

    start_time = time.time()
    targetBoxes = detector.detect_bbox([img])
    detect_bbox_time = time.time() - start_time
    
    start_time = time.time()
    all_points = npPointsCraft.detect(img, targetBoxes)
    all_points = [ps for ps in all_points if len(ps)]
    craft_time = time.time() - start_time
    
    start_time = time.time()
    
    zones = convertCvZonesRGBtoBGR([getCvZoneRGB(img, reshapePoints(rect,1)) for rect in all_points])
    
    perspective_align_time = time.time() - start_time
    
    start_time = time.time()
    regionIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    classification_time = time.time() - start_time
    
    start_time = time.time()
    textArr = textDetector.predict(zones, regionNames, countLines)
    ocr_time = time.time() - start_time
    return (image_load_time, detect_bbox_time, craft_time, perspective_align_time, classification_time, ocr_time)

In [14]:
N = 10

i = 0
j = 0

image_load_time_all        = 0
detect_bbox_time_all       = 0
craft_time_all             = 0
perspective_align_time_all = 0
classification_time_all    = 0
ocr_time_all               = 0

start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            image_load_time, \
                detect_bbox_time, \
                craft_time, \
                perspective_align_time, \
                classification_time, \
                ocr_time = await test(dirName, fname)
            image_load_time_all        += image_load_time
            detect_bbox_time_all       += detect_bbox_time
            craft_time_all             += craft_time
            perspective_align_time_all += perspective_align_time
            classification_time_all    += classification_time
            ocr_time_all               += ocr_time
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9


### Jetson Xavier Tensorrt

In [15]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 310 photos
Time 112.0932343006134
One photo process 0.3615910783890755 seconds

image_load_time_all 13.902168989181519; 0.04484570641671458 per one photo
detect_bbox_time_all 9.671597003936768; 0.03119870001269925 per one photo
craft_time_all 71.47682309150696; 0.2305703970693773 per one photo
perspective_align_time_all 4.427069902420044; 0.014280870652967884 per one photo
classification_time_all 3.8065600395202637; 0.012279225933936334 per one photo
ocr_time_all 8.722760915756226; 0.02813793843792331 per one photo


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + RTX 3090 Tensorrt in Docker

In [15]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 310 photos
Time 19.011606216430664
One photo process 0.06132776198848601 seconds

image_load_time_all 5.987424373626709; 0.019314272172989384 per one photo
detect_bbox_time_all 2.3786933422088623; 0.007673204329706008 per one photo
craft_time_all 7.732449054718018; 0.024943384047477476 per one photo
perspective_align_time_all 0.9697110652923584; 0.003128100210620511 per one photo
classification_time_all 0.6272952556610107; 0.002023533082777454 per one photo
ocr_time_all 1.298546314239502; 0.004188859078191942 per one photo
